In [1]:
import ipywidgets as widgets
from IPython.display import display, Markdown, Javascript, clear_output 
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from functools import partial
import json
import requests
from ipywidgets import interact
from ipywidgets import Button, HBox, VBox, HTML, Label

In [2]:
initial_setup = {
    "pipeline_id": "2",
    "version": 1.0,
    "stages": [
        {
            "task_group_id": "TAG1",
            "branch": [
                {
                    "task_id": "Task_Name_XX",
                    "code_file_path": "tasks/base_creation/final_base_logic.hql",
                    "language": "Hive",
                    "config": {
                        "f": "TE1"
                    },
                    "sequence": 0,
                    "condition": "in_start_date in range [2021-10-01 , 2023-11-04]"
                }
            ],
            "default": {
                "task_id": "TAG1_default",
                "code_file_path": "tasks/base_creation/default_base_logic.hql",
                "language": "Hive",
                "config": {}
            }
        },
        {
            "task_group_id": "TAG2",
            "branch": [
                {
                    "task_id": "Task_Name_XX",
                    "code_file_path": "tasks/variables_creation/final_cas_logic.hql",
                    "language": "Hive",
                    "config": {
                        "gE5": "a"
                    },
                    "sequence": 1,
                    "condition": "in_start_date in range [2022-02-01 , 2023-11-04]"
                },
                {
                    "task_id": "Task_Name_XX",
                    "code_file_path": "tasks/variables_creation/final_sor_logic.hql",
                    "language": "Hive",
                    "config": {
                        "gE6": "tE6"
                    },
                    "sequence": 2,
                    "condition": "in_start_date in range [2021-10-01 , 2022-01-31]"
                }
            ],
            "default": {
                "task_id": "TAG2_default",
                "code_file_path": "tasks/variables_creation/default_variables_logic.hql",
                "language": "Hive",
                "config": {}
            }
        },
        {
            "task_id": "simple_task_1",
            "tag": "2",
            "code_file_path": "tasks/base_task.py",
            "language": "pyspark",
            "config": {
              }
        },
        {
            "task_id": "simple_task_2",
            "tag": "3",
            "code_file_path": "tasks/base_task.py",
            "language": "pyspark",
            "config": {
              }
        },
        {
            "task_id": "aggregation_logic",
            "code_file_path": "tasks/aggregation/aggregation_logic_mvp.py",
            "language": "PySpark",
            "config": {}
        }
    ],
    "dependencies": " ['task_group_id_01_Name >> task_group_id_02_Name >> aggregation_logic']",
    "pipeline_type": "new_feature",
    "deploy_type": "analytical"
}

In [3]:
count_group = 0
for i in range(len(initial_setup['stages'])):
    
    for j in initial_setup['stages'][i].keys():
        if j == 'task_group_id':
            count_group = count_group + 1
count_group

2

In [4]:
vars_config = []
vars_config_widgets = []
output_widgets = []

In [5]:
# for removing any task info
def remove_task(task_index):
    for i in range(len(output_widgets)):
        with output_widgets[i]:
            clear_output()

    del output_widgets[task_index]
    del vars_config[task_index]
    del vars_config_widgets[task_index]
    new_task_name.value = f'task_group_{len(vars_config)+1}'
    display_all_tasks()

In [6]:

# vars_config## [{'task_group_id': 'simple_task_1', 'code_file_path': 'tasks/base_task.py',
# 'language': 'pyspark', 'config': {'spark.driver.memory': '15432m'}}, 
#        {'task_group_id': 'simple_task_2', 'code_file_path': 'tasks/base_task.py', 'language': 'pyspark', 'config': {}}]

In [7]:
def add_new_task(b):

    vars_config.append({'task_group_id': new_task_name.value,'code_file_path': '','config': {},'language': 'hive','num_params':1 })
    vars_config_widgets.append({})
    output_widgets.append(widgets.Output())
    with parent_output:
        display(output_widgets[-1])

    new_task_name.value = f'task_group_{len(vars_config)+1}'
    display_task(len(vars_config)-1)

In [8]:
def display_task(task_index):
    # clearing out the previous values in the widgets array
    vars_config_widgets[task_index] = {}
    task_widget_array = []
    i = task_index
    with output_widgets[i]:
        clear_output()
## GDChange2
        print('vars_config##',vars_config)
        vars_config_widgets[i]['task_group_id'] = widgets.Text(value=vars_config[i]['task_group_id'],
                                                               description='simple task Name:',
                                                               style={'description_width': 'initial'})
        task_widget_array.append(vars_config_widgets[i]['task_group_id'])
        
        vars_config_widgets[i]['code_file_path'] = widgets.Text(value=vars_config[i]['code_file_path'],
                                                                description='code_file_path:',
                                                                style={'description_width': 'initial'})
        task_widget_array.append(vars_config_widgets[i]['code_file_path'])
        
        vars_config_widgets[i]['language'] = widgets.Dropdown(options=['hive', 'pyspark'], 
                                                              value=vars_config[i]['language'], 
                                                              description='language :', 
                                                              style={'description_width': 'initial'})
        task_widget_array.append(vars_config_widgets[i]['language'])
        
        
        vars_config_widgets[i]['num_params'] = widgets.IntText(value=vars_config[i]['num_params'],
                                                            description='num_params:',
                                                            style={'description_width': 'initial'})
                                                            
        task_widget_array.append(vars_config_widgets[i]['num_params'])
        length = vars_config_widgets[i]['num_params'].value

        for i in range(length):

            vars_config_widgets[i]['config'] = HBox([widgets.Text(value='2', description='Key', style={
                                    'description_width': 'initial'},),
            widgets.Text(value='2', description='value', style={
                                    'description_width': 'initial'},)])

        # vars_config_widgets[i]['config'] = widgets.Text(value='',
        #                                                         description='config:',
        #                                                         style={'description_width': 'initial'})
            task_widget_array.append(vars_config_widgets[i]['config'])
        print('var_config_widget', vars_config_widgets[i]['num_params'].value)
        print('vars_config_widgets##',vars_config_widgets)
        print('task_widget_array##',task_widget_array)
        
## TDchange1x
#         for j in vars_config[i]:
#             task_tw_widgets_array = []
#         task_widget_array.append(widgets.Label())

        vars_config_widgets[i]['delete'] = widgets.Button(
            description='Delete Task Group', button_style='danger')
        def delete_task(b):
            remove_task(task_index)
        vars_config_widgets[i]['delete'].on_click(delete_task)
        task_widget_array.append(vars_config_widgets[i]['delete'])
        vars_config_widgets[i]['accordion'] = widgets.Accordion(children=[
            widgets.VBox(task_widget_array)])
        vars_config_widgets[i]['accordion'].set_title(
            0, vars_config[i]['task_group_id'])
        display(vars_config_widgets[i]['accordion'])
        display(Markdown('''---'''))

def display_all_tasks():
    for ind in range(len(vars_config)):
        display_task(ind)


In [9]:
vars_config

[]

In [10]:
vars_config_widgets

[]

In [11]:
add_task_bt = widgets.Button(
    description='+ Add Task Group', button_style='primary')
new_task_name = widgets.Text(value=f'task_group_{len(output_widgets)+1}')
output_add_bt = widgets.Output()
display(output_add_bt)
with output_add_bt:
    display(widgets.HBox([new_task_name, add_task_bt]))
    add_task_bt.on_click(add_new_task)

Output()

In [12]:
# vars_config_widgets[0]['config']['key'] = widgets.Text(value=task_tw_config[0][0],
#                                                             description='Key',
#                                                             style={'description_width': 'initial'})

In [13]:
# task_widget_array## 
# [Text(value='simple_task_1', description='simple task Name:', style=DescriptionStyle(description_width='initial')), 
# Text(value='tasks/base_task.py', description='code_file_path:', style=DescriptionStyle(description_width='initial')), 
# Dropdown(description='language :', index=1, options=('hive', 'pyspark'), 
#          style=DescriptionStyle(description_width='initial'), value='pyspark'), 
# IntText(value=0, description='num_params:', style=DescriptionStyle(description_width='initial'))]

In [14]:
parent_output = widgets.Output()
display(parent_output)

Output()

Output()

Output()

In [15]:
# vars_config[0]['config']

In [16]:
# vars_config = [{'task_group_id': 'simple_task_1',
#   'code_file_path': 'tasks/base_task.py',
#   'language': 'pyspark',
#   'config': {'spark.driver.memory': '15432m'},
#   'num_params': 1},
#  {'task_group_id': 'simple_task_2',
#   'code_file_path': 'tasks/base_task.py',
#   'language': 'pyspark',
#   'config': {},
#   'num_params': 0}]

In [17]:
# i =2,3


In [18]:
def initialize_form():
    for i in range(count_group,len(initial_setup['stages'])-1):
        output_widgets.append(widgets.Output())
        vars_config_widgets.append({})
        ## GDChange1
        vars_config.append({'task_group_id': list(initial_setup['stages'][i].values())[0],
                            'code_file_path': initial_setup['stages'][i]['code_file_path'],
                            'language': initial_setup['stages'][i]['language'],
                            'config': initial_setup['stages'][i]['config'],
                           'num_params': 1,})

#         print('vars_config',vars_config)

    with parent_output:
        for i in range(len(output_widgets)):
            display(output_widgets[i])
        

    display_all_tasks()

initialize_form()